# Exercise 2 - Interactive charting with plotly

## 0. Internet usage

In [4]:
# a) Recreate this visualization using plotly, make it as close as possible to the image.

import pandas as pd

df_internet = pd.read_csv("data/share-of-individuals-using-the-internet.csv")
df_internet.head()

,Entity,Code,Year,Individuals using the Internet (% of population)
0,Afghanistan,AFG,1990,0.0
1,Afghanistan,AFG,1991,0.0
2,Afghanistan,AFG,1992,0.0
3,Afghanistan,AFG,1993,0.0
4,Afghanistan,AFG,1994,0.0


In [5]:
import duckdb

df_selection = duckdb.query("""--sql
    SELECT 
        Entity,
        Year,
        "Individuals using the Internet (% of population)"
    FROM df_internet
    WHERE Entity IN (
                    'East Asia and Pacific (WB)',
                    'Europe and Central Asia (WB)', 
                    'Latin America and Caribbean (WB)',
                    'Middle East and North Africa (WB)',
                    'North America (WB)',
                    'South Asia (WB)',
                    'Sub-Saharan Africa (WB)',
                    'World'  )
""").to_df()

df_selection

,Entity,Year,Individuals using the Internet (% of population)
0,Europe and Central Asia (WB),1990,0.048031
1,Europe and Central Asia (WB),1991,0.101868
2,Europe and Central Asia (WB),1992,0.180602
3,Europe and Central Asia (WB),1993,0.279651
4,Europe and Central Asia (WB),1994,0.524235
...,...,...,...
265,World,2019,53.200000
266,World,2020,59.300000
267,World,2021,62.200000
268,World,2022,64.400000


In [6]:
df_visualize_old = duckdb.query("""--sql
    SELECT 
        Entity,
        ROUND("Individuals using the Internet (% of population)",1)
    FROM df_selection
    WHERE Year IN (2021, 2023)
    ORDER BY "Individuals using the Internet (% of population)" DESC
""").to_df()         

df_visualize_old

,Entity,"round(""Individuals using the Internet (% of population)"", 1)"
0,North America (WB),97.3
1,North America (WB),96.4
2,Europe and Central Asia (WB),90.1
3,Europe and Central Asia (WB),86.0
4,Latin America and Caribbean (WB),81.0
5,East Asia and Pacific (WB),79.0
6,Middle East and North Africa (WB),77.7
7,Latin America and Caribbean (WB),76.0
8,East Asia and Pacific (WB),72.8
9,Middle East and North Africa (WB),72.6


In [7]:
df_visualize = duckdb.query("""--sql
    WITH ranked_data AS (
        SELECT 
            Entity,
            "Individuals using the Internet (% of population)",
            Year,
            ROW_NUMBER() OVER (PARTITION BY Entity ORDER BY Year DESC) AS rn
        FROM df_selection
        WHERE Year IN (2021, 2023)
    )
    SELECT 
        Entity,
        ROUND("Individuals using the Internet (% of population)",1) AS "Individuals using the Internet (% of population)"
    FROM ranked_data
    WHERE rn = 1
    ORDER BY "Individuals using the Internet (% of population)" ASC
""").to_df()

df_visualize


,Entity,Individuals using the Internet (% of population)
0,Sub-Saharan Africa (WB),36.7
1,South Asia (WB),42.9
2,World,67.4
3,Middle East and North Africa (WB),77.7
4,East Asia and Pacific (WB),79.0
5,Latin America and Caribbean (WB),81.0
6,Europe and Central Asia (WB),90.1
7,North America (WB),97.3


In [74]:
import plotly.express as px

bar_color = "#7088B0"
top_title = "#2D2E2D"
sub_title = "#5B5B5B"
y_label_color = "#555555"
explanations_color = "#858585"

# Plot horizontal bar chart
fig = px.bar(
    df_visualize,
    x="Individuals using the Internet (% of population)",
    y="Entity",
    orientation="h",
    color_discrete_sequence=[bar_color],
    text="Individuals using the Internet (% of population)",
)

fig.update_traces(
    textposition="outside",  # place values at end of each bar
    texttemplate="%{text:.1f}%",  # format as percentage with 1 decimal
    textfont=dict(size=14, color="#444444"),
    cliponaxis=False)



fig.update_layout(
    xaxis=dict(showticklabels=False),  # optional: hide axis ticks
    margin=dict(r=100),  # give space for text
    plot_bgcolor="white",
    paper_bgcolor="white"
)


# Update layout with title, font, size, color, and position, as well as removing labels
fig.update_layout(
    title=dict(
        text=(
            "<span style='font-size:30px; color:top_title;font-weight:bold;'>"
            "Share of the population using the Internet, 2023"
            "</span><br><br>"
            "<span style='font-size:24px; color:sub_title;'>"
            "Share of the population who used the Internet<sup>1</sup> in the last three months."
            "</span>"
        ),
        x=0.01,  # Position title 
        y=0.9,
        xanchor="left"  
    ),
    xaxis_title=None,
    yaxis_title=None,
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        tickfont=dict(size=16, color=y_label_color, weight= "bold"),# family="Lato"
        domain=[0, 0.85],
    ),
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False, 
    ),
)

fig.add_annotation(
    text=(
        "<span style='font-size:16px; color:" + explanations_color + ";'>"
        "<b>Data source:</b> International Telecommunication Union (via World Bank) (2025)"
        "                              OurWorldinData.org/internet | CC BY"
        "</span>"
    ),
    xref="paper",  # Relative to the plot width (0 to 1)
    yref="paper",  # Relative to the plot height (0 to 1)
    x=- 0.4,        # Match title x position (left-aligned)
    y=-0.06,       # Below the plot (adjust as needed)
    showarrow=False,
    align="left"
)

# Separation line
fig.add_shape(
    type="line",
    xref="paper",
    yref="paper",
    x0=-0.4,
    x1=0.99,
    y0=-0.08,
    y1=-0.08,
    line=dict(color=explanations_color, width=1)
)

# Additional explanatory text
fig.add_annotation(
    text="<span style='font-size:15px; color:" + explanations_color + ";'>"
         "<b>1. Internet user:</b>An internet user is defined by the International Telecommunication Union as anyone who has accessed the internet from any<br>"
         "location in the last three months. This can be from any type of device, including a computer, mobile phone, personal digital asssistant, games<br>"
         "machine, digital TV, and other technological devices.</span>",
    xref="paper",
    yref="paper",
    x=-0.4,
    y=-0.17,
    showarrow=False,
    align="left"
)
fig.add_layout_image(
    dict(
        source="figures/logo.png",  
        xref="paper", yref="paper",  # Relative to figure, not data
        x=1.05, y=.95,                  # Bottom-right corner
        sizex=0.12, sizey=0.12,      # Adjust size as needed
        xanchor="right", yanchor="bottom",
        layer="above"               # Draw above plot
    )
)

# Ensure enough space at bottom
fig.update_layout(margin=dict(b=120,r=80))

fig.update_layout(paper_bgcolor="white", plot_bgcolor="white", hovermode=False)
fig.update_layout(width=1200, height=800)


# Show the plot
fig.show()
